<a href="https://colab.research.google.com/github/mc-friday/hanghaeAI/blob/main/%5B2%EC%A3%BC%EC%B0%A8%5D%EA%B8%B0%EB%B3%B8%EA%B3%BC%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [2주차] 기본과제: 주어진 문장에서 나올 다음 단어를 예측하는 모델 구현

In [80]:
!pip install datasets sacremoses

In [81]:
import torch
from datasets import load_dataset
from torch.utils.data import DataLoader
from transformers import BertTokenizerFast
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)
from torch.nn.utils.rnn import pad_sequence
from torch.optim import Adam
import numpy as np
from math import sqrt
import matplotlib.pyplot as plt
from torch import nn
import time

## 1. 데이터셋 (기존의 IMDB dataset을 그대로 활용)

In [ ]:
ds = load_dataset("stanfordnlp/imdb")
tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'bert-base-uncased')


### 1.1. [MY_CODE] Last word prediction dataset 준비를 위한 함수

In [ ]:
def collate_fn(batch):
    max_len = 400
    texts, labels = [], []
    for row in batch:
        input_ids = tokenizer(row['text'], truncation=True, max_length=max_len).input_ids
        if len(input_ids) > 2: #[MYCODE]input_ids[:-2]로 마지막 두 개의 토큰을 제거하는데, 길이가 2 이하라면 결과적으로 빈 입력이라서 추가
            labels.append(input_ids[-2])  # 과제적용 : 마지막 두 번째 토큰
            texts.append(torch.LongTensor(input_ids[:-2]))  # 과제적용 : 마지막 두 토큰 제외
    texts = pad_sequence(texts, batch_first=True, padding_value=tokenizer.pad_token_id)
    labels = torch.LongTensor(labels)
    return texts, labels

### 1.2. DataLoader 생성

In [ ]:
train_loader = DataLoader(
    ds['train'], batch_size=64, shuffle=True, collate_fn=collate_fn
)
test_loader = DataLoader(
    ds['test'], batch_size=64, shuffle=False, collate_fn=collate_fn
)

## 2. 모델 구현

### 2.1. SelfAttention Model

self-attention

In [ ]:
class SelfAttention(nn.Module):
  def __init__(self, input_dim, d_model):
    super().__init__()

    self.input_dim = input_dim
    self.d_model = d_model

    self.wq = nn.Linear(input_dim, d_model)
    self.wk = nn.Linear(input_dim, d_model)
    self.wv = nn.Linear(input_dim, d_model)
    self.dense = nn.Linear(d_model, d_model)

    self.softmax = nn.Softmax(dim=-1)

  def forward(self, x, mask):
    q, k, v = self.wq(x), self.wk(x), self.wv(x)
    score = torch.matmul(q, k.transpose(-1, -2)) # (B, S, D) * (B, D, S) = (B, S, S)
    score = score / sqrt(self.d_model)

    if mask is not None:
      score = score + (mask * -1e9)

    score = self.softmax(score)
    result = torch.matmul(score, v)
    result = self.dense(result)

    return result

### 2.2. TransformerLayer Model

self-attention과 feed-forward layer를 구현한 모습

In [ ]:
class TransformerLayer(nn.Module):
  def __init__(self, input_dim, d_model, dff):
    super().__init__()

    self.input_dim = input_dim
    self.d_model = d_model
    self.dff = dff

    self.sa = SelfAttention(input_dim, d_model)
    self.ffn = nn.Sequential(
      nn.Linear(d_model, dff),
      nn.ReLU(),
      nn.Linear(dff, d_model)
    )

  def forward(self, x, mask):
    x = self.sa(x, mask)
    x = self.ffn(x)

    return x

### 2.3. [MY_CODE] TextClassifier Model

TransformerLayer와 positional encoding을 모두 합친 모습

In [ ]:
class TextClassifier(nn.Module):
  def __init__(self, vocab_size, d_model, n_layers, dff):
    super().__init__()

    self.vocab_size = vocab_size
    self.d_model = d_model
    self.n_layers = n_layers
    self.dff = dff

    self.embedding = nn.Embedding(vocab_size, d_model)
    self.pos_encoding = nn.parameter.Parameter(positional_encoding(max_len, d_model), requires_grad=False)
    self.layers = nn.ModuleList([TransformerLayer(d_model, d_model, dff) for _ in range(n_layers)])
    self.classification = nn.Linear(d_model, vocab_size) #[MYCODE] 과제적용 출력차원을 1 -> vocab_size

  def forward(self, x):
    mask = (x == tokenizer.pad_token_id)
    mask = mask[:, None, :]
    seq_len = x.shape[1]

    x = self.embedding(x)
    x = x * sqrt(self.d_model)
    x = x + self.pos_encoding[:, :seq_len]

    for layer in self.layers:
      x = layer(x, mask)

    x = x[:, -1]
    x = self.classification(x)

    return x


## 3. Positional encoding

In [ ]:
def get_angles(pos, i, d_model):
    angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
    return pos * angle_rates

def positional_encoding(position, d_model):
    angle_rads = get_angles(np.arange(position)[:, None], np.arange(d_model)[None, :], d_model)
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
    pos_encoding = angle_rads[None, ...]

    return torch.FloatTensor(pos_encoding)


max_len = 400
print(f"[LOG] {positional_encoding(max_len, 256).shape}")

## 4. [MY_CODE] 모델, 손실 함수, 옵티마이저 설정

In [ ]:
model = TextClassifier(len(tokenizer), 32, 2, 32)
lr = 0.001
model = model.to('cuda')
loss_fn = nn.CrossEntropyLoss() #[MYCODE] 과제적용 : 손실 함수 변경 이진 분류 -> 다중 클래스 분류

optimizer = Adam(model.parameters(), lr=lr)

### 4.1. [MY_CODE] model의 정확도를 측정하는 함수

In [ ]:
def accuracy(model, dataloader):
  model.eval()
  cnt = 0
  acc = 0

  for data in dataloader:
    inputs, labels = data
    inputs, labels = inputs.to('cuda'), labels.to('cuda')

    preds = model(inputs)
    preds = torch.argmax(preds, dim=-1) #[MYCODE] 과제적용 : 손실 함수 변경 이진 분류 -> 다중 클래스 분류

    cnt += labels.shape[0]
    acc += (preds == labels).sum().item()

  return acc / cnt

### 4.2. [MY_CODE] plot 함수

In [ ]:
def plot_acc(train_accs, test_accs, title, label1='train', label2='test'):
    """
    학습 정확도 및 테스트 정확도를 시각화하는 함수
    Args:
        train_accs: 각 epoch별 학습 정확도 리스트
        test_accs: 각 epoch별 테스트 정확도 리스트
        title: 그래프 제목
        label1: 학습 데이터 라벨
        label2: 테스트 데이터 라벨
    """
    plt.figure(figsize=(10, 5))
    x = np.arange(len(train_accs))
    plt.plot(x, train_accs, label=label1)
    plt.plot(x, test_accs, label=label2)
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.legend()
    plt.title(title)
    plt.show()

## 5. [MY_CODE] 학습

In [ ]:
# [MYCODE] 학습 루프에 정확도 및 손실 저장 추가
n_epochs = 50
train_acc_list = []
test_acc_list = []
train_loss_list = []

for epoch in range(n_epochs):
  epoch_start = time.time() #[MYCODE] 학습 시간 측정 시작
  total_loss = 0.
  model.train()
  for data in train_loader:
    model.zero_grad()
    inputs, labels = data
    inputs, labels = inputs.to('cuda'), labels.to('cuda').float()

    preds = model(inputs)
    loss = loss_fn(preds, labels)
    loss.backward()
    optimizer.step()

    total_loss += loss.item()

  epoch_time = time.time() - epoch_start #[MYCODE] 학습 시간 측정 종료
  print(f"[LOG] Epoch {epoch + 1} | Train Loss: {total_loss:.4f} | Time: {epoch_time:.2f}s")

  eval_start_time = time.time()  #[MYCODE] 평가 시간 측정 시작

  with torch.no_grad():
    model.eval()
    train_acc = accuracy(model, train_loader)
    test_acc = accuracy(model, test_loader)

  eval_time = time.time() - eval_start_time  #[MYCODE] 평가 시간 측정 종료
  print(f"[LOG] =========> Train acc: {train_acc:.3f} | Test acc: {test_acc:.3f} | Time: {epoch_time:.2f}s")

  # [MYCODE] 정확도 저장
  train_acc_list.append(train_acc)
  test_acc_list.append(test_acc)

## 5.1 [MY_CODE] 그래프 출력

In [ ]:
# [MYCODE] 학습 완료 후 그래프 출력
plot_acc(train_acc_list, test_acc_list, title="Train vs Test Accuracy")